In [1]:
import dateutil
import os

import cv2
import torch
import torchvision.ops.boxes as bops
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import clear_output, display

In [2]:
def intersect(box1, box2):
    """
    Determines whether an OpenCV bounding box intersects another bounding box.
    Used for determining when an ice cloud region is in contact with a rainy region
    and thus precipitating.

    Args:
        box1: first bounding box
        box2: second bounding box

    Returns:
        intersects: boolean on whether the boxes intersect
    """
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    torch_box1 = torch.tensor(np.array([[x1, y1, x1 + w1, y1 + h1]]), dtype=torch.float)
    torch_box2 = torch.tensor(np.array([[x2, y2, x2 + w2, y2 + h2]]), dtype=torch.float)
    iou = bops.box_iou(torch_box1, torch_box2)
    return not not iou.any()

def longest_edge(contour):
    """
    Calculates the longest straight edge of a contour.
    Value can be used to determine whether a significant section of the cloud has been cut off
    by attenuation or border of the plot which affects how crinkly the perimeter is.

    Args:
        contour: contour around the cloud
    
    Returns:
        longest_edge: longest straight edge of the cloud
    """
    prev_diff = None
    length = 0
    max_length = 0
    for i in range(len(contour) - 1):
        diff = contour[i] - contour[i + 1]
        if (diff == prev_diff).all():
            length += 1
        else:
            length = 1
        prev_diff = diff
        max_length = max(length, max_length)
    return max_length

def get_contours(img_hsv, img_gray, hsv_values):
    """
    Find contours by thresholding images with HSV values.

    Args:
        img_hsv: image in HSV format
        img_gray: grayscale image
        hsv_values: values to determine what colour to find contours around

    Returns:
        contours: contours found
    """
    mask = cv2.inRange(img_hsv, hsv_values, hsv_values)
    img = cv2.bitwise_and(img_gray, img_gray, mask=mask)
    img[img != 0] = 255
    _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(image=thresh, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
    return contours

def find_clouds(file_name):
    """
    Finds clouds that fulfill certain criteria which will be used to calculate fractal dimension.

    Args:
        file_name: name of image file to find clouds in

    Returns:
        img: the image read form the file with contours drawn
        contours_ice: contours of all valid ice clouds
    """
    img = cv2.imread(file_name)

    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask_drop = cv2.inRange(img_hsv, (119, 159, 185), (119, 159, 185))
    img[:, :1317, :][mask_drop[:, :1317] > 0] = (187, 176, 160)
    
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    contours_drizzle = get_contours(img_hsv, img_gray, (102, 221, 243))
    contours_melt = get_contours(img_hsv, img_gray, (19, 255, 255))
    contours_ice = get_contours(img_hsv, img_gray, (102, 37, 187))

    contours_ice = [contour for contour in contours_ice if cv2.contourArea(contour) > 0]
    contours_melt = [contour for contour in contours_melt if cv2.contourArea(contour) > 0]
    contours_drizzle = [contour for contour in contours_drizzle if cv2.contourArea(contour) > 0]
    
    box_melt = []
    box_ice = []
    box_drizzle = []
    box_corrupt = []
    to_remove = []
    
    for contour in contours_ice:
        x, y, w, h = cv2.boundingRect(contour)
        if y < 42 or x > 1317:
            box_corrupt.append([x, y, w, h])
        box_ice.append([x, y, w, h])
    
    for contour in contours_melt:
        x, y, w, h = cv2.boundingRect(contour)
        y -= 2
        box_melt.append([x, y, w, h])

    for contour in contours_drizzle:
        x, y, w, h = cv2.boundingRect(contour)
        y -= 2
        box_drizzle.append([x, y, w, h])
    
    for i, v in enumerate(box_ice):
        if longest_edge(contours_ice[i]) / cv2.arcLength(contours_ice[i], True) > 0.3:
            to_remove.append(i)
        else:
            for j in box_melt + box_drizzle + box_corrupt:
                if intersect(v, j):
                    to_remove.append(i)
                    break
    
    for i in sorted(to_remove, reverse=True):
        contours_ice.pop(i)
    
    cv2.drawContours(image=img, contours=contours_ice, contourIdx=-1, color=(0, 255, 0), thickness=1, lineType=cv2.LINE_AA)
    return img, contours_ice

In [3]:
files = os.listdir('./cloudnet-collection')
first_file = min(files).split('_')
sites = {file.split('_')[1] for file in files}

back = widgets.Button(description='Back')
next = widgets.Button(description='Next')
dropdown = widgets.Dropdown(description='Site:', options=[(site.capitalize(), site) for site in sites], value=first_file[1])
datepicker = widgets.DatePicker(description='Date:', value=dateutil.parser.parse(first_file[0]))
slider = widgets.IntSlider(description='Contour:')
output = widgets.Output()

count = 0
contours = None
img = None

display(widgets.HBox([back, next]), dropdown, datepicker, slider, output)

def navigate(button):
    global count
    count = count - 1 if button.description == 'Back' else count + 1
    date, site, _ = files[count].split('_')
    date = dateutil.parser.parse(date)
    if datepicker.value != date and dropdown.value != site:
        datepicker.unobserve(value_change, names='value')
        datepicker.value = date
        dropdown.value = site
        datepicker.observe(value_change, names='value')
    else:
        datepicker.value = date
        dropdown.value = site

def show_image():
    global count, contours, img, slider
    back.disabled = count == 0
    next.disabled = count == len(files) - 1
    img, contours = find_clouds('./cloudnet-collection/' + files[count])
    slider.min, slider.max = -1, len(contours)
    slider.value = -1
    if len(contours) > 0:
        slider.disabled = False
        slider.min = 1
        slider.value = 1
    else:
        slider.disabled = True
        slider.min = 0
        slider.value = 0

def value_change(_):
    global count
    site = dropdown.value
    date = datepicker.value
    if date:
        date_str = date.strftime('%Y%m%d')
        try:
            count = files.index(f'{date_str}_{site}_classification.png')
            show_image()
        except:
            with output:
                clear_output()
                print(f'No data at {site.capitalize()} on {date.strftime("%d/%m/%Y")}')
    else:
        with output:
            clear_output()
            print('Select a date')
    
def contour_change(_):
    global contours, img, slider
    if slider.value >= 0:
        with output:
            clear_output()
            img_copy = img.copy()
            img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB)
            if len(contours) > 0:
                cv2.drawContours(image=img_copy, contours=[contours[slider.value - 1]], contourIdx=-1, color=(255, 0, 0), thickness=1, lineType=cv2.LINE_AA)
                print(f'Perimeter (px): {cv2.arcLength(contours[slider.value - 1], True)}')
                print(f'Area (px): {cv2.contourArea(contours[slider.value - 1])}')
                x, y, _, h = cv2.boundingRect(contours[slider.value - 1])
                x = (x - 72) / 1245 * 24
                y = (410 - y - h) / 371 * 12
                print(f'Coordinates (x, y): {x, y}')
            else:
                print('No valid clouds')
            plt.figure(figsize=(15, 15))
            plt.axis('off')
            plt.imshow(img_copy)
            plt.show()

back.on_click(navigate)
next.on_click(navigate)
dropdown.observe(value_change, names='value')
datepicker.observe(value_change, names='value')
slider.observe(contour_change, names='value')
show_image()

Dropdown(description='Site:', index=3, options=(('Munich', 'munich'), ('Galati', 'galati'), ('Lindenberg', 'li…

DatePicker(value=datetime.datetime(1999, 5, 1, 0, 0), description='Date:', step=1)

IntSlider(value=0, description='Contour:')

Output()